# 1C Customers dynamic features


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
import pandas as pd

sys.path.append("..")

from src.utils.core_utils import setup_logging

# Initialize logging
file_log = "customer_dynamic_feature.log"
root_logger = setup_logging(level=logging.DEBUG, log_file=file_log, remove_existing=True)

2025-05-31 19:10:32,128 - src.utils - INFO - Creating file handler with level: 10
2025-05-31 19:10:32,129 - src.utils - DEBUG - Logging setup complete to customer_dynamic_feature.log


In [3]:
from src.feature_customers import CustomerDynamicFeaturePipelineConfig, CustomerDynamicFeaturePipeline

/Users/sharonshiao/Dropbox/Machine_learning/projects/fahsion_recommendation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Run pipeline


In [4]:
customer_dynamic_feature_config = CustomerDynamicFeaturePipelineConfig.create_default()

In [5]:
customer_dynamic_feature_pipeline = CustomerDynamicFeaturePipeline(customer_dynamic_feature_config)

In [6]:
customer_dynamic_feature_pipeline.setup()

2025-05-31 19:10:34,882 - src.feature_customers - INFO - Setting up CustomerDynamicFeaturePipeline
2025-05-31 19:10:34,883 - src.feature_customers - DEBUG - Config: {
  "config_processor": {
    "start_week_num": 76,
    "end_week_num": 102,
    "k_items": 5
  },
  "subsample": 0.05,
  "seed": 42
}


In [7]:
results_customer_dynamic_feature = customer_dynamic_feature_pipeline.run()

2025-05-31 19:10:34,903 - src.feature_customers - INFO - Loading data for CustomerDynamicFeaturePipeline
2025-05-31 19:10:34,904 - src.feature_extraction - INFO - Loading optimized raw data from transactions train 0.05 42
2025-05-31 19:10:34,906 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_train_sample_0.05_42.parquet
2025-05-31 19:10:35,037 - src.feature_extraction - INFO - Loading optimized raw data from transactions valid 0.05 42
2025-05-31 19:10:35,039 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_valid_sample_0.05_42.parquet
2025-05-31 19:10:35,043 - src.feature_extraction - INFO - Loading optimized raw data from transactions test 0.05 42
2025-05-31 19:10:35,044 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_test_sample_0.05_42.parquet
2025-05-31 19:10:35,063 - src.features_articles - INFO - Loading article embedding result from ../data/preprocessed/articles_embedding/full
2025-05-31 19:10:35,210 -

# Checks


In [8]:
results_customer_dynamic_feature.data.info()
print(results_customer_dynamic_feature.data.shape)
display(results_customer_dynamic_feature.data.head())
print(results_customer_dynamic_feature.feature_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012203 entries, 0 to 1012202
Data columns (total 5 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   customer_id                   1012203 non-null  uint64 
 1   week_num                      1012203 non-null  int64  
 2   customer_avg_price            1012203 non-null  float64
 3   customer_avg_text_embedding   647807 non-null   object 
 4   customer_avg_image_embedding  647807 non-null   object 
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 38.6+ MB
(1012203, 5)


,customer_id,week_num,customer_avg_price,customer_avg_text_embedding,customer_avg_image_embedding
0,345001598676045,76,0.028990,NaN,NaN
1,345001598676045,77,0.029267,NaN,NaN
2,345001598676045,78,0.029543,NaN,NaN
3,345001598676045,79,0.029896,NaN,NaN
4,345001598676045,80,0.029613,NaN,NaN


{'numerical_features': ['customer_avg_price'], 'embedding_features': ['customer_avg_text_embedding', 'customer_avg_image_embedding'], 'one_hot_features': [], 'categorical_features': [], 'id_columns': ['customer_id', 'week_num']}
